In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

retiros=pd.read_csv("/content/Retiros_historicos.csv")


In [2]:
retiros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108878 entries, 0 to 108877
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID_EMPLEADOS              108878 non-null  int64  
 1   COMPAÑÍA                  108878 non-null  object 
 2   CODIGO_AREA               108878 non-null  int64  
 3   AREA                      108867 non-null  object 
 4   CODIGO_CARGO              108878 non-null  int64  
 5   CARGO                     108878 non-null  object 
 6   REGION                    108878 non-null  object 
 7   FORMA_DE_TRABAJO          108878 non-null  object 
 8   CLASE_DE_MEDIDA           108878 non-null  object 
 9   MOTIVO_RETIRO             108878 non-null  object 
 10  FECHA_DESDE               108878 non-null  object 
 11  CODIGO_AREA_ANTERIOR      90372 non-null   float64
 12  CODIGO_CARGO_ANTERIOR     86299 non-null   float64
 13  AÑO                       108878 non-null  i

In [3]:
pd.set_option("display.max_columns",None)              #con este código podemos observar todas las columnas del df.
retiros.sample(2)

,ID_EMPLEADOS,COMPAÑÍA,CODIGO_AREA,AREA,CODIGO_CARGO,CARGO,REGION,FORMA_DE_TRABAJO,CLASE_DE_MEDIDA,MOTIVO_RETIRO,FECHA_DESDE,CODIGO_AREA_ANTERIOR,CODIGO_CARGO_ANTERIOR,AÑO,PERIODO,VP ACTUAL,VP ANT,2 ACT,2 ANT,3 ACT,3 ANT,4 ACT,4 ANT,5 ACT,5 ANT,NIVEL_DEL_CARGO,RED_DE_SUCURSALES,CARGO_ANTERIOR,COMPAÑÍA_AANTERIOR,NIVEL_DEL_CARGO_ANTERIOR,GÉNERO
36532,2883,Bancolombia S.A.,1669,SUC LAURELES DS,21758,CAJERO/A,Antioquia,Convencionados (CO),CAMBIO ORG. SIN CAM. REM (CO),TRASLADO HORIZONTAL,3/01/2022,1448.0,3060.0,2022,ene-22,VICEPRESIDENCIA DE NEGOCIOS,VICEPRESIDENCIA DE NEGOCIOS,VICEPRESIDENCIA NEGOCIOS PERSN PYME EMP,VICEPRESIDENCIA NEGOCIOS PERSN PYME EMP,VICEPRESIDENCIA PERSONAS Y PYME ANTIOQUI,VICEPRESIDENCIA PERSONAS Y PYME ANTIOQUI,GCIA ZONA OCCIDENTE BPP ANTIOQUIA,GCIA ZONA OCCIDENTE BPP ANTIOQUIA,SUC BELEN,SUC BELEN,OPERATIVO,RED DE SUCURSALES,CAJERO/A,Bancolombia S.A.,OPERATIVO,femenino
53788,48547,Bancolombia S.A.,2451,GCIA IMPLEMENTACION DE SERV ESTRAT 2,92306,PROFESIONAL DE PROYECTOS,Dirección gener,No Convencionad (CO),CONTRATACION (CO),CONTRATACIÓN,9/09/2019,NaN,NaN,2019,sep-19,VICEPRESIDENCIA SERVICIOS CORPORATIVOS,NaN,VICEPRESIDENCIA SERV CLIENTES EMPLEADOS,NaN,CDE PROCESOS,NaN,ADC AYD HABILITADORES ADMON FINANCIERA,NaN,LDC FC HABILITADORES DE INFORMACION,NaN,PROFESIONALES,GRUPO SIN RED DE SUCURSALES,ANALISTA III,NaN,NaN,Femenino


In [4]:
retiros["CLASE_DE_MEDIDA"].value_counts()     #Valores unicos por clase de medida



,count
CLASE_DE_MEDIDA,
CAMBIO ORG. SIN CAM. REM (CO),37699
CAMBIO ORG. CON CAM. REM (CO),32588
CONTRATACION (CO),18494
RETIRO (CO),16174
IMPACTO SALARIAL (CO),3922
RECONTRATACION (CO),1


In [5]:
retiros["MOTIVO_RETIRO"].value_counts()            #Valores unicos por motivo de retiro

,count
MOTIVO_RETIRO,
TRASLADO HORIZONTAL,32896
PROMOCIÓN,30437
CONTRATACIÓN,18488
VOLUNTARIO ARREGLO MUTUO,6149
RETIRO VOLUNTARIO,5180
TRASLADO POR CAMBIO DE OFICINA,3466
SUSTITUCION PATRONAL INTERNA,2118
MOD. POR CAMBIO DE JORNADA,1935
RETIRO SIN JUSTA CAUSA,1819


In [6]:
output = []
for col in retiros.columns:
    vc = retiros[col].value_counts(dropna=False).sort_index()
    total = vc.sum()
    output.append(f"{col}:\n{vc}\nTotal: {total}")
print('\n\n'.join(output))                                              #con esta formula se verifica todos los valores unicos a la vez, permitiendo visualizar si tenemos valores nulos, caracteres especiales en todas las columnas a la vez

ID_EMPLEADOS:
ID_EMPLEADOS
12          1
30          1
34          3
41          3
42          1
           ..
90062236    1
90062248    1
90062257    1
90062315    1
92000929    1
Name: count, Length: 38566, dtype: int64
Total: 108878

COMPAÑÍA:
COMPAÑÍA
Banca de Inv. Bancolombia       565
Bancolombia S.A.             103024
Factoring Bancolombia S.A        33
Fiduciaria Bancolombia         2813
Leasing Bancolombia S.A.        707
Valores Bancolombia S.A.       1736
Name: count, dtype: int64
Total: 108878

CODIGO_AREA:
CODIGO_AREA
0           11
3            8
4           31
6           12
14          12
            ..
70006498     1
70006499     1
70006500     1
70006501     1
70006507     1
Name: count, Length: 4640, dtype: int64
Total: 108878

AREA:
AREA
 GCIA MERCADEO DE CANALES                    7
 GERENCIA COMERCIAL DE INCLUSION             2
 GERENCIA ESTRATEGIA CCIAL DE INCLUSION      2
 GERENCIA MERCADEO PERSONAS                  7
ADC ADMIN DE IDENTIDADES Y ACCESOS         

# **VALORES NULOS, CARACTERES ESPECIALES Y EN BLANCO**

Se revisa y se identifica que las siguientes columnas no presentan carácteres especiales, ni en blancos, más sin embargo, se identifican algunos valores nulos. Para este ejercicio determinamos que las columnas con un porcentaje de valores nulos mayor al 30% será eliminado.  veamos:
1. Area= 11
2. Còdigo de area= 2
3. Còdigo de cargo= 22579***20%
4. VP actual= 12
5. Vp Ant= 18543***17%
6. 2 act= 12
7. 2 ant= 18543***17%
8. 3 act= 12
9. 3 ant= 18543***17%
10. 4 act= 12
11. 4 ant= 18543***17%
12. 5 act= 12
13. 5 ant= 18543***17%
14. Nivel de cargo= 3387***3%
15. red de sucursales= 3387***3%
16. Cargo anterior= 7201***7%
17. Compañia anterior= 27928***26%
18. Nivel de cargo anterior= 18769***17%


Se analizan cada una de las columnas y se determina que no se presentan valores en blanco, ni carácteres especiales. Más sin embargo, se identifican algunos valores nulos que representan menos del 30%, por lo que por este motivo no será eliminadas.


# **LIMPIEZA**

# VARIABLE GENERO
Se identifica que la columna: "Género presenta inconsistencia en sus datos, se escribe de diferentes formas, por lo que se procede a limpiar la columna a continuación.

In [7]:
retiros['GÉNERO'] = retiros['GÉNERO'].str.strip().str.lower()   #.str.strip() Este método elimina los espacios en blanco al inicio y al final de cada valor de texto en la columna.

                                                                #.str.lower() Después de quitar espacios, este método convierte todo el texto a minúsculas.
retiros['GÉNERO'] = retiros['GÉNERO'].replace({
    'femenino': 'Femenino',
    'feménino': 'Femenino',
    'masculino': 'Masculino',
    'masculíno': 'Masculino',
})


print(retiros['GÉNERO'].unique())

['Masculino' 'Femenino']


# VARIABLE NIVEL DE CARGO
Se identifica que la columna: "Nivel de cargo" presenta inconsistencia en sus datos, se escribe de diferentes formas, por lo que se procede a limpiar la columna a continuación.

In [8]:
retiros['NIVEL_DEL_CARGO'] = retiros['NIVEL_DEL_CARGO'].replace('PROFESIONALES', 'PROFESIONAL')


In [9]:
retiros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108878 entries, 0 to 108877
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID_EMPLEADOS              108878 non-null  int64  
 1   COMPAÑÍA                  108878 non-null  object 
 2   CODIGO_AREA               108878 non-null  int64  
 3   AREA                      108867 non-null  object 
 4   CODIGO_CARGO              108878 non-null  int64  
 5   CARGO                     108878 non-null  object 
 6   REGION                    108878 non-null  object 
 7   FORMA_DE_TRABAJO          108878 non-null  object 
 8   CLASE_DE_MEDIDA           108878 non-null  object 
 9   MOTIVO_RETIRO             108878 non-null  object 
 10  FECHA_DESDE               108878 non-null  object 
 11  CODIGO_AREA_ANTERIOR      90372 non-null   float64
 12  CODIGO_CARGO_ANTERIOR     86299 non-null   float64
 13  AÑO                       108878 non-null  i

Se revisa el tipo de datos de las columnas a trabajar en el modelo y se identifica que todas presentan el formato correcto; más sin embargo, se identifica que la variable año, es pertienente convertirla a categorica, ya que con este dato no se pueden sacar estadisticas. Y, la columna: periodo se debe convertir a: DATATIME, ya que es una fecha, además se identifica la necesidad de crear una nueva columna, para la variable a predecir, en este caso: "Retiros voluntarios"

# **CREAR COLUMNA PARA RETIROS VOLUNTARIOS Y RETIROS VOLUNTARIOS POR ARREGLO MUTUO"**

Vamos  a clasificar la variable a predecir como "RETIRO VOLUNTARIO" en donde sus valores van a ser:

1 --> Verdadero: El empleado se retiro voluntariamente (en este valor van a estar relacionados de la variable MOTIVO  DE RETIRO, los valores de "RETIRO VOLUNTARIO" y "VOLUNTARIO ARREGLO MUTUO"

0 --> Falso: El empelado se retiro de su cargo por cualquier otro motivo  los cuales estan relacionados en la variable MOTIVO  DE RETIRO ≠ "RETIRO VOLUNTARIO" y "VOLUNTARIO ARREGLO MUTUO"

In [10]:
retiros['RETIRO_VOLUNTARIO'] = np.where(
    retiros['MOTIVO_RETIRO'].isin(['RETIRO VOLUNTARIO', 'VOLUNTARIO ARREGLO MUTUO']),
    1,
    0
)

In [11]:
print(retiros['RETIRO_VOLUNTARIO'].value_counts().sort_index())
print("Suma total frecuencia:", retiros['RETIRO_VOLUNTARIO'].value_counts(dropna=False).sum())

RETIRO_VOLUNTARIO
0    97549
1    11329
Name: count, dtype: int64
Suma total frecuencia: 108878


In [12]:
retiros.sample(2)

,ID_EMPLEADOS,COMPAÑÍA,CODIGO_AREA,AREA,CODIGO_CARGO,CARGO,REGION,FORMA_DE_TRABAJO,CLASE_DE_MEDIDA,MOTIVO_RETIRO,FECHA_DESDE,CODIGO_AREA_ANTERIOR,CODIGO_CARGO_ANTERIOR,AÑO,PERIODO,VP ACTUAL,VP ANT,2 ACT,2 ANT,3 ACT,3 ANT,4 ACT,4 ANT,5 ACT,5 ANT,NIVEL_DEL_CARGO,RED_DE_SUCURSALES,CARGO_ANTERIOR,COMPAÑÍA_AANTERIOR,NIVEL_DEL_CARGO_ANTERIOR,GÉNERO,RETIRO_VOLUNTARIO
97147,19260,Valores Bancolombia S.A.,1122,MESA TASA DE INTERES,99999999,TRADER SENIOR,Dirección gener,No Convencionad (CO),RETIRO (CO),VOLUNTARIO ARREGLO MUTUO,3/03/2015,1122.0,47701.0,2015,mar-15,VICEPRESIDENCIA CORP GRUPO BANCOLOMBIA,VICEPRESIDENCIA CORP GRUPO BANCOLOMBIA,VICEPRESIDENCIA FINANCIERA,VICEPRESIDENCIA FINANCIERA,VICEPRESIDENCIA TESORERIA,VICEPRESIDENCIA TESORERIA,POSICION PROPIA,POSICION PROPIA,MESA TASA DE INTERES,MESA TASA DE INTERES,PROFESIONAL,GRUPO SIN RED DE SUCURSALES,TRADER SENIOR,Valores Bancolombia S.A.,PROFESIONALES,Masculino,1
34178,39007,Bancolombia S.A.,184,SUC COPACABANA,21231,ASESOR/A COMERCIAL MULTISEGMENTO,Antioquia,Convencionados (CO),CAMBIO ORG. SIN CAM. REM (CO),TRASLADO HORIZONTAL,1/03/2022,162.0,14758.0,2022,mar-22,VICEPRESIDENCIA DE NEGOCIOS,VICEPRESIDENCIA DE NEGOCIOS,VICEPRESIDENCIA NEGOCIOS PERSN PYME EMP,VICEPRESIDENCIA NEGOCIOS PERSN PYME EMP,VICEPRESIDENCIA PERSONAS Y PYME ANTIOQUI,VICEPRESIDENCIA PERSONAS Y PYME ANTIOQUI,GCIA ZONA NORTE Y CHOCO BPP ANTIOQUIA,GCIA ZONA NORTE Y CHOCO BPP ANTIOQUIA,SUC BELLO,SUC BELLO,OPERATIVO,RED DE SUCURSALES,ASESOR/A COMERCIAL MULTISEGMENTO,Bancolombia S.A.,OPERATIVO,Masculino,0


In [13]:
retiros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108878 entries, 0 to 108877
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID_EMPLEADOS              108878 non-null  int64  
 1   COMPAÑÍA                  108878 non-null  object 
 2   CODIGO_AREA               108878 non-null  int64  
 3   AREA                      108867 non-null  object 
 4   CODIGO_CARGO              108878 non-null  int64  
 5   CARGO                     108878 non-null  object 
 6   REGION                    108878 non-null  object 
 7   FORMA_DE_TRABAJO          108878 non-null  object 
 8   CLASE_DE_MEDIDA           108878 non-null  object 
 9   MOTIVO_RETIRO             108878 non-null  object 
 10  FECHA_DESDE               108878 non-null  object 
 11  CODIGO_AREA_ANTERIOR      90372 non-null   float64
 12  CODIGO_CARGO_ANTERIOR     86299 non-null   float64
 13  AÑO                       108878 non-null  i

# ESTADISTICA DESCRIPTIVA

In [14]:
variables_modelo = [
    "COMPAÑÍA", "CODIGO_AREA", "AREA", "CODIGO_CARGO", "CARGO", "REGION",
    "FORMA_DE_TRABAJO", "AÑO", "PERIODO", "GÉNERO", "RETIRO_VOLUNTARIO", "NIVEL_DEL_CARGO","RED_DE_SUCURSALES"
]

In [15]:
retiros_reducido = retiros[variables_modelo].copy()

Convertimos 'RETIRO_VOLUNTARIO' como BOOLEANA para poderlo representar mejor en un diagrama de barras

In [16]:
retiros['RETIRO_VOLUNTARIO'] = retiros['RETIRO_VOLUNTARIO'].astype("bool")

In [17]:
import plotly.express as px

# Reemplazar True/False por "Sí"/"No"
conteo = retiros_reducido['RETIRO_VOLUNTARIO'].replace({True: "Sí", False: "No"}).value_counts().reset_index()
conteo.columns = ['RETIRO_VOLUNTARIO', 'count']

# Crear gráfico de barras con etiquetas visibles
fig = px.bar(
    conteo,
    x='RETIRO_VOLUNTARIO',
    y='count',
    text='count',
    labels={'RETIRO_VOLUNTARIO': '¿Se retiró voluntariamente?', 'count': 'Cantidad'},
    title='Retiros voluntarios en Bancolombia',
    width=600,
    height=500  # Aumenta altura para dar más espacio
)

# Posición y estilo del texto
fig.update_traces(
    textposition='outside',
    textfont_size=14  # Tamaño más grande para mejor lectura
)

# Ajustar márgenes y diseño
fig.update_layout(
    margin=dict(t=100),  # Espacio superior extra
    yaxis_title='Cantidad',
    xaxis_title='¿Se retiró voluntariamente?',
    uniformtext_minsize=8,
    uniformtext_mode='show'
)

fig.show()

In [18]:
import plotly.express as px

# Paso 1: Top 20 códigos de área
top_areas = retiros['CODIGO_AREA'].value_counts().head(20).index

# Paso 2: Filtrar los datos
df_top = retiros[retiros['CODIGO_AREA'].isin(top_areas)]

# Paso 3: Agrupar por código de área y retiro voluntario
grouped = df_top.groupby(['CODIGO_AREA', 'RETIRO_VOLUNTARIO'], observed=True).size().reset_index(name='Cantidad')

# Paso 4: Convertir booleanos a texto
grouped['RETIRO_VOLUNTARIO'] = grouped['RETIRO_VOLUNTARIO'].map({False: 'No', True: 'Sí'})

# Paso 5: Asegurar que el código de área sea string
grouped['CODIGO_AREA'] = grouped['CODIGO_AREA'].astype(str)

# Paso 6: Crear el gráfico
fig = px.bar(
    grouped,
    x='CODIGO_AREA',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Códigos de Área vs Retiro Voluntario',
    labels={
        'CODIGO_AREA': 'Código de Área',
        'Cantidad': 'Número de Retiros',
        'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'
    },
    barmode='stack'
)

# Paso 7: Ajustar el ángulo de las etiquetas
fig.update_layout(xaxis_tickangle=-45)

# Mostrar el gráfico
fig.show()


In [19]:
import plotly.express as px

# Paso 1: Top 20 áreas por cantidad de retiros
top_areas = retiros_reducido['AREA'].value_counts().head(20).index

# Paso 2: Filtrar el DataFrame por las top 20 áreas
df_top = retiros_reducido[retiros_reducido['AREA'].isin(top_areas)]

# Paso 3: Agrupar por AREA y RETIRO_VOLUNTARIO
grouped = df_top.groupby(['AREA', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 4: Asegurar que AREA esté como string (por si acaso)
grouped['AREA'] = grouped['AREA'].astype(str)

# Paso 5: Crear el gráfico apilado
fig = px.bar(
    grouped,
    x='AREA',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Áreas vs Retiro Voluntario',
    labels={
        'AREA': 'Área',
        'Cantidad': 'Número de Retiros',
        'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'
    },
    barmode='stack'  # Apilado
)

# Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1500,  # Ancho del gráfico
    height=800   # Alto del gráfico
)

fig.show()


In [20]:
import plotly.express as px

# Paso 1: Top 20 códigos de cargo
top_cargos = retiros_reducido['CODIGO_CARGO'].value_counts().head(20).index

# Paso 2: Filtrar el DataFrame por los top 20 códigos de cargo
df_top = retiros_reducido[retiros_reducido['CODIGO_CARGO'].isin(top_cargos)]

# Paso 3: Agrupar por código de cargo y retiro voluntario
grouped = df_top.groupby(['CODIGO_CARGO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 4: Asegurar que CODIGO_CARGO esté como string para que sea categórico en el eje X
grouped['CODIGO_CARGO'] = grouped['CODIGO_CARGO'].astype(str)

# Paso 5: Crear el gráfico apilado
fig = px.bar(
    grouped,
    x='CODIGO_CARGO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Códigos de Cargo vs Retiro Voluntario',
    labels={'CODIGO_CARGO': 'Código de Cargo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack'  # Apilado
)

# Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1500,  # Ancho del gráfico
    height=800   # Alto del gráfico
)

fig.show()

In [21]:
# Mostrar la tabla con el top 20 de códigos de cargo y retiro voluntario
tabla_top20 = grouped.sort_values(by='Cantidad', ascending=False)
display(tabla_top20.sort_index())


,CODIGO_CARGO,RETIRO_VOLUNTARIO,Cantidad
0,24763,0,27
1,24834,0,24
2,25366,0,44
3,47121,0,33
4,63474,0,20
5,64149,0,20
6,71160,0,32
7,71181,0,21
8,74803,0,23
9,74956,0,29


In [22]:
import plotly.express as px

# Paso 1: Top 20 cargos por cantidad de retiros
top_cargos = retiros_reducido['CARGO'].value_counts().head(20).index

# Paso 2: Filtrar el DataFrame por los top 20 cargos
df_top = retiros_reducido[retiros_reducido['CARGO'].isin(top_cargos)]

# Paso 3: Agrupar por CARGO y RETIRO_VOLUNTARIO
grouped = df_top.groupby(['CARGO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 4: Asegurar que CARGO esté como string para que sea categórico en el eje X
grouped['CARGO'] = grouped['CARGO'].astype(str)

# Paso 5: Crear el gráfico apilado
fig = px.bar(
    grouped,
    x='CARGO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Cargos vs Retiro Voluntario',
    labels={'CARGO': 'Cargo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack'  # Apilado
)

# Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1500,  # Ancho
    height=800   # Alto
)

fig.show()

In [23]:
import pandas as pd
import plotly.express as px
import numpy as np

# Paso 1: Reemplazar los nombres de los meses con su formato en inglés
month_translation = {
    'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',
    'jul': 'Jul', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
}

# Asegurarse de que los valores de la columna 'PERIODO' estén en minúsculas
retiros_reducido['PERIODO'] = retiros_reducido['PERIODO'].str.lower()

# Reemplazar los meses con su formato en inglés
retiros_reducido['PERIODO'] = retiros_reducido['PERIODO'].replace(month_translation, regex=True)

# Paso 2: Convertir la columna PERIODO a formato datetime (ahora con meses en inglés)
retiros_reducido['PERIODO'] = pd.to_datetime(retiros_reducido['PERIODO'], format='%b-%y', errors='coerce')

# Paso 3: Verificar si hay valores nulos después de la conversión
print(retiros_reducido[retiros_reducido['PERIODO'].isna()])

# Paso 4: Calcular el número de bins según la regla de Freedman-Diaconis
Q1 = retiros_reducido['PERIODO'].quantile(0.25)
Q3 = retiros_reducido['PERIODO'].quantile(0.75)
IQR = Q3 - Q1
bin_width = 2 * IQR / (len(retiros_reducido['PERIODO']) ** (1 / 3))
bins = int((retiros_reducido['PERIODO'].max() - retiros_reducido['PERIODO'].min()) / bin_width)

# Paso 5: Crear el gráfico apilado con la variable RETIRO_VOLUNTARIO
fig = px.histogram(
    retiros_reducido,
    x='PERIODO',
    color='RETIRO_VOLUNTARIO',  # Apilar según RETIRO_VOLUNTARIO
    nbins=bins,  # Número de bins calculado
    title='Histograma de Frecuencia de Periodo por Retiro Voluntario',
    labels={'PERIODO': 'Periodo', 'count': 'Frecuencia'},
    barmode='stack',  # Apilado
    color_discrete_map={  # Asignar colores específicos
        False: 'blue',    # Falso será azul
        True: 'orange'    # Verdadero será naranja
    }
)

# Paso 6: Agregar contornos a las barras y modificar el tamaño del gráfico
fig.update_traces(marker_line_width=2, marker_line_color='black')  # Contornos de las barras en negro

# Paso 7: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1000,  # Cambiar el ancho según desees
    height=600   # Cambiar la altura según desees
)

# Mostrar el gráfico
fig.show()

Empty DataFrame
Columns: [COMPAÑÍA, CODIGO_AREA, AREA, CODIGO_CARGO, CARGO, REGION, FORMA_DE_TRABAJO, AÑO, PERIODO, GÉNERO, RETIRO_VOLUNTARIO, NIVEL_DEL_CARGO, RED_DE_SUCURSALES]
Index: []


In [24]:
import pandas as pd
import plotly.express as px

# Paso 1: Agrupar por la variable COMPAÑÍA y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['COMPAÑÍA', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 2: Asegurarse de que la columna COMPAÑÍA esté como string (si es necesario)
grouped['COMPAÑÍA'] = grouped['COMPAÑÍA'].astype(str)

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='COMPAÑÍA',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Compañía y Retiro Voluntario',
    labels={'COMPAÑÍA': 'Compañía', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1600,  # Ancho del gráfico
    height=900   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()


In [25]:
import pandas as pd
import plotly.express as px

# Paso 1: Agrupar por la variable REGION y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['REGION', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 2: Asegurarse de que la columna REGION esté como string (si es necesario)
grouped['REGION'] = grouped['REGION'].astype(str)

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='REGION',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Región y Retiro Voluntario',
    labels={'REGION': 'Región', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()


Pasos para un Feature Engineering

In [26]:
import pandas as pd
import plotly.express as px

# Paso 1: Agrupar por la variable FORMA_DE_TRABAJO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['FORMA_DE_TRABAJO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 2: Asegurarse de que la columna FORMA_DE_TRABAJO esté como string (si es necesario)
grouped['FORMA_DE_TRABAJO'] = grouped['FORMA_DE_TRABAJO'].astype(str)

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='FORMA_DE_TRABAJO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Forma de Trabajo y Retiro Voluntario',
    labels={'FORMA_DE_TRABAJO': 'Forma de Trabajo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()


In [27]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna AÑO sea de tipo string (si es necesario)
retiros_reducido['AÑO'] = retiros_reducido['AÑO'].astype(str)

# Paso 2: Agrupar por la variable AÑO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['AÑO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='AÑO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Año y Retiro Voluntario',
    labels={'AÑO': 'Año', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [28]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna GÉNERO sea de tipo string (si es necesario)
retiros_reducido['GÉNERO'] = retiros_reducido['GÉNERO'].astype(str)

# Paso 2: Agrupar por la variable GÉNERO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['GÉNERO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='GÉNERO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Género y Retiro Voluntario',
    labels={'GÉNERO': 'Género', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [29]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna NIVEL_DEL_CARGO sea de tipo string (si es necesario)
retiros_reducido['NIVEL_DEL_CARGO'] = retiros_reducido['NIVEL_DEL_CARGO'].astype(str)

# Paso 2: Agrupar por la variable NIVEL_DEL_CARGO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['NIVEL_DEL_CARGO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='NIVEL_DEL_CARGO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Nivel del Cargo y Retiro Voluntario',
    labels={'NIVEL_DEL_CARGO': 'Nivel del Cargo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [30]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna RED_DE_SUCURSALES sea de tipo string (si es necesario)
retiros_reducido['RED_DE_SUCURSALES'] = retiros_reducido['RED_DE_SUCURSALES'].astype(str)

# Paso 2: Agrupar por la variable RED_DE_SUCURSALES y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['RED_DE_SUCURSALES', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='RED_DE_SUCURSALES',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Red de Sucursales y Retiro Voluntario',
    labels={'RED_DE_SUCURSALES': 'Red de Sucursales', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [31]:
# base libraries for data science
from pathlib import Path

import pandas as pd
import sklearn as sk
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [32]:
retiros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108878 entries, 0 to 108877
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID_EMPLEADOS              108878 non-null  int64  
 1   COMPAÑÍA                  108878 non-null  object 
 2   CODIGO_AREA               108878 non-null  int64  
 3   AREA                      108867 non-null  object 
 4   CODIGO_CARGO              108878 non-null  int64  
 5   CARGO                     108878 non-null  object 
 6   REGION                    108878 non-null  object 
 7   FORMA_DE_TRABAJO          108878 non-null  object 
 8   CLASE_DE_MEDIDA           108878 non-null  object 
 9   MOTIVO_RETIRO             108878 non-null  object 
 10  FECHA_DESDE               108878 non-null  object 
 11  CODIGO_AREA_ANTERIOR      90372 non-null   float64
 12  CODIGO_CARGO_ANTERIOR     86299 non-null   float64
 13  AÑO                       108878 non-null  i

ELIMINACIÓN DE COLUMNAS NO NECESARIAS PARA EL MODELO

In [33]:
variables_modelo = [
    "COMPAÑÍA", "CODIGO_AREA", "AREA", "CODIGO_CARGO", "CARGO", "REGION",
    "FORMA_DE_TRABAJO", "AÑO", "PERIODO", "GÉNERO", "RETIRO_VOLUNTARIO", "NIVEL_DEL_CARGO","RED_DE_SUCURSALES"
]

Estas variables describen al empleado antes del retiro y son apropiadas para entrenar el modelo:

COMPAÑÍA, CODIGO_AREA, AREA, CODIGO_CARGO, CARGO, REGION, FORMA_DE_TRABAJO, AÑO, PERIODO, GÉNERO.

El resto de variables se eliminan de la variable retiro_reducido, por qué no aporta información pertinente para el modelo o estas variables reflejan la decisión o consecuencia del retiro, o están muy relacionadas con la definición directa del target, por lo que confundiria el modelo.

In [34]:
retiros_reducido = retiros[variables_modelo].copy()
retiros_reducido

,COMPAÑÍA,CODIGO_AREA,AREA,CODIGO_CARGO,CARGO,REGION,FORMA_DE_TRABAJO,AÑO,PERIODO,GÉNERO,RETIRO_VOLUNTARIO,NIVEL_DEL_CARGO,RED_DE_SUCURSALES
0,Bancolombia S.A.,70005072,SUC ARMENIA CENTRO CCIAL 1,99999999,Integración: Posición por defecto,Sur,Convencionados (CO),2025,mar-25,Masculino,True,NaN,NaN
1,Bancolombia S.A.,264,GCIA AUD PERSONAS PYMES Y EMPRESAS,10542,AUDITOR/A,Antioquia,No Convencionad (CO),2025,mar-25,Masculino,False,PROFESIONAL,GRUPO SIN RED DE SUCURSALES
2,Bancolombia S.A.,2738,SUC BANCA COLOMBIA OVIEDO DS,99999999,Integración: Posición por defecto,Antioquia,No Convencionad (CO),2025,mar-25,Femenino,True,NaN,NaN
3,Bancolombia S.A.,70005179,SUC CALASANZ CCIAL 1,99999999,Integración: Posición por defecto,Antioquia,Convencionados (CO),2025,mar-25,Femenino,True,NaN,NaN
4,Bancolombia S.A.,1005,SUC PUERTO LOPEZ,99999999,Integración: Posición por defecto,Centro,Convencionados (CO),2025,mar-25,Femenino,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108873,Bancolombia S.A.,3364,GCIA ESTRATEGIA DE PROCESOS,15156,ANALISTA I,Dirección gener,No Convencionad (CO),2014,ene-14,Femenino,False,PROFESIONAL,GRUPO SIN RED DE SUCURSALES
108874,Bancolombia S.A.,906,SUC OLIMPICA,7431,ASESOR INTEGRAL II,Caribe,Convencionados (CO),2014,ene-14,Femenino,False,OPERATIVO,RED DE SUCURSALES
108875,Bancolombia S.A.,1571,SUC CUCUTA DS,99999999,DIRECTOR SERVICIO AL CLIENTE,Centro,En transición (CO),2014,ene-14,Femenino,False,JEFE,RED DE SUCURSALES
108876,Bancolombia S.A.,581,CENTRO GESTION DOCUMENTAL ANTIOQUIA,99999999,AUX IMAGENES I,Antioquia,Convencionados (CO),2014,ene-14,Femenino,False,OPERATIVO,GRUPO SIN RED DE SUCURSALES


In [35]:
retiros_reducido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108878 entries, 0 to 108877
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   COMPAÑÍA           108878 non-null  object
 1   CODIGO_AREA        108878 non-null  int64 
 2   AREA               108867 non-null  object
 3   CODIGO_CARGO       108878 non-null  int64 
 4   CARGO              108878 non-null  object
 5   REGION             108878 non-null  object
 6   FORMA_DE_TRABAJO   108878 non-null  object
 7   AÑO                108878 non-null  int64 
 8   PERIODO            108878 non-null  object
 9   GÉNERO             108878 non-null  object
 10  RETIRO_VOLUNTARIO  108878 non-null  bool  
 11  NIVEL_DEL_CARGO    105491 non-null  object
 12  RED_DE_SUCURSALES  105491 non-null  object
dtypes: bool(1), int64(3), object(9)
memory usage: 10.1+ MB


VALORES FALTANTES

In [36]:
retiros_reducido.isna().sum()

,0
COMPAÑÍA,0
CODIGO_AREA,0
AREA,11
CODIGO_CARGO,0
CARGO,0
REGION,0
FORMA_DE_TRABAJO,0
AÑO,0
PERIODO,0
GÉNERO,0


DATOS DUPLICADOS

In [37]:
filas_duplicadas = retiros_reducido.duplicated().sum()
print("Cantidad de filas duplicadas: ", filas_duplicadas)

Cantidad de filas duplicadas:  1850


Se identifica que al renombrar el dataframe y manejar solo las variables necesarias para el modelo, tenemos información duplicada, pero esto se debe a que se elimina el ID; por lo que es normal que se repitan las filas con datos comunes entre sí.

In [38]:
retiros_reducido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108878 entries, 0 to 108877
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   COMPAÑÍA           108878 non-null  object
 1   CODIGO_AREA        108878 non-null  int64 
 2   AREA               108867 non-null  object
 3   CODIGO_CARGO       108878 non-null  int64 
 4   CARGO              108878 non-null  object
 5   REGION             108878 non-null  object
 6   FORMA_DE_TRABAJO   108878 non-null  object
 7   AÑO                108878 non-null  int64 
 8   PERIODO            108878 non-null  object
 9   GÉNERO             108878 non-null  object
 10  RETIRO_VOLUNTARIO  108878 non-null  bool  
 11  NIVEL_DEL_CARGO    105491 non-null  object
 12  RED_DE_SUCURSALES  105491 non-null  object
dtypes: bool(1), int64(3), object(9)
memory usage: 10.1+ MB


INGENIERIA DE DATOS

In [39]:
# Codificar la variable objetivo

retiros_reducido["RETIRO_VOLUNTARIO"] = retiros_reducido["RETIRO_VOLUNTARIO"].astype("int")               #.astype, se utiliza para cambiar el tipo de dato

retiros_reducido.sample(5)

,COMPAÑÍA,CODIGO_AREA,AREA,CODIGO_CARGO,CARGO,REGION,FORMA_DE_TRABAJO,AÑO,PERIODO,GÉNERO,RETIRO_VOLUNTARIO,NIVEL_DEL_CARGO,RED_DE_SUCURSALES
68157,Bancolombia S.A.,2543,SECCION ADMON Y SERVICIO CANAL ANTIOQUIA,15653,AUX INTEGRAL DE SERVICIOS,Dirección gener,Convencionados (CO),2017,oct-17,Masculino,0,OPERATIVO,GRUPO SIN RED DE SUCURSALES
67145,Bancolombia S.A.,2909,CELULA PRESTANOMINA 1,30570,LIDER CELULA,Dirección gener,No Convencionad (CO),2017,dic-17,Masculino,0,PROFESIONAL,GRUPO SIN RED DE SUCURSALES
30345,Bancolombia S.A.,804,SUC UNICENTRO PASTO,70117786,ASESOR/A COMERCIAL MULTISEGMENTO,Sur,Convencionados (CO),2022,jun-22,Femenino,0,OPERATIVO,RED DE SUCURSALES
59960,Bancolombia S.A.,4993,GCIA DE IMPLEMENTACIONES CONTABLES,78766,ANALISTA III,Dirección gener,No Convencionad (CO),2018,dic-18,Femenino,0,PROFESIONAL,GRUPO SIN RED DE SUCURSALES
6546,Bancolombia S.A.,6676,GCIA DE RELACION CORP 8,9225,GERENTE/A RELACION,Bogota y sabana,No Convencionad (CO),2024,ago-24,Femenino,0,ESTRATEGICO MEDIO,GRUPO SIN RED DE SUCURSALES


In [40]:
# Categoricas ordinales y target

target = 'RETIRO_VOLUNTARIO'

cols_categoric_ordinal = ["AÑO"]
print(cols_categoric_ordinal)

['AÑO']


In [41]:
# Columnas categóricas (no ordenadas)

cols_categoric_nominales = retiros_reducido.select_dtypes(include=['object', 'category']).columns.tolist()   #se puede hacer con el código de las ordinales o con este, llamando directamente los object y categoricos
cols_categoric_nominales

['COMPAÑÍA',
 'AREA',
 'CARGO',
 'REGION',
 'FORMA_DE_TRABAJO',
 'PERIODO',
 'GÉNERO',
 'NIVEL_DEL_CARGO',
 'RED_DE_SUCURSALES']

In [42]:
# Columnas fecha
cols_fecha = retiros_reducido.select_dtypes(include=['datetime64[ns]']).columns.tolist()
cols_fecha

[]

# **PIPELINES**

In [43]:
categorical_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder()),
    ]
)

In [44]:
categorical_ord_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder()),
    ]
)

# **PREPROCESADOR**

In [45]:
preprocessor = ColumnTransformer(
    transformers=[        ("categoric", categorical_pipe, cols_categoric_nominales),
        ("categoric ordinales", categorical_ord_pipe, cols_categoric_ordinal),
    ]
)

preprocessor

ColumnTransformer(transformers=[('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['COMPAÑÍA', 'AREA', 'CARGO', 'REGION',
                                  'FORMA_DE_TRABAJO', 'PERIODO', 'GÉNERO',
                                  'NIVEL_DEL_CARGO', 'RED_DE_SUCURSALES']),
                                ('categoric ordinales',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['AÑO'])])

# **Train / Test split**

In [46]:
# conjunto de entrenamiento y de prueba

X_features = retiros_reducido.drop("RETIRO_VOLUNTARIO", axis="columns")
Y_target = retiros_reducido["RETIRO_VOLUNTARIO"]

In [47]:
# 80% train, 20% test

x_train, x_test, y_train, y_test = train_test_split(
    X_features, Y_target, test_size=0.2, stratify=Y_target
)

In [48]:
# Verificamos dimensiones

x_train.shape, y_train.shape

((87102, 12), (87102,))

In [49]:
# Verificamos dimensiones

x_test.shape, y_test.shape

((21776, 12), (21776,))

# **Preprocesamiento**

In [50]:
transformed_data = preprocessor.fit(x_train)

transformed_data

ColumnTransformer(transformers=[('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['COMPAÑÍA', 'AREA', 'CARGO', 'REGION',
                                  'FORMA_DE_TRABAJO', 'PERIODO', 'GÉNERO',
                                  'NIVEL_DEL_CARGO', 'RED_DE_SUCURSALES']),
                                ('categoric ordinales',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['AÑO'])])

In [51]:
feature_names = preprocessor.get_feature_names_out()

feature_names

array(['categoric__COMPAÑÍA_Banca de Inv. Bancolombia',
       'categoric__COMPAÑÍA_Bancolombia S.A.',
       'categoric__COMPAÑÍA_Factoring Bancolombia S.A', ...,
       'categoric__RED_DE_SUCURSALES_GRUPO SIN RED DE SUCURSALES',
       'categoric__RED_DE_SUCURSALES_RED DE SUCURSALES',
       'categoric ordinales__AÑO'], dtype=object)

Para ver con más detalle las transformaciones del one hote encoding, podemos convertir rápidamente en un dataframe las variables preprocesadas: